In [1]:
import os
import matplotlib.pyplot as plt
import glob
import rasterio
from rasterio.merge import merge
from rasterio.plot import show
from rasterio.mask import mask
from rasterstats import zonal_stats
import fiona
import geopandas as gpd
import xarray as xr
import numpy as np
import rioxarray
import pandas as pd
import dask

# Set all nan values to np.nan for parameters (only need to run once)

In [ ]:
def reassign_mask_npnan(param_dirs):
    
    for folder in param_dirs:
        files=np.array(sorted(os.listdir(folder)))
        tifs = pd.Series(files).str.contains('.tif')
        files = files[tifs]
        print(folder)
        
        for filename in files:
            with rasterio.open(folder+filename, 'r+') as ds:
                print(filename)
                a = ds.read()# read all raster values
                a[a < 0 ] = np.nan  
                ds.write(a)

In [ ]:
reassign_mask_npnan(all_dirs)

# Create Land Cover Masks

In [ ]:
def create_land_mask(in_dir, out_dir, lc_num):
    files=np.array(sorted(os.listdir(in_dir)))
    
    for filename in files:
        base = os.path.splitext(filename)[0]
        print(base)
        rast = rasterio.open(in_dir+filename)
        meta = rast.meta
        meta.update(dtype = 'float32', nodata = -3.4e+38)
        rast.close()
        
        with rasterio.open(out_dir+base+'_' + str(lc_num) + '.tif', 'w', **meta) as dst:
            with rasterio.open(in_dir+filename) as src:
                data = src.read()
                data = data.astype(np.float32)
                data[data!=lc_num] = np.nan
                data[data==lc_num] = 1
                dst.write(data)

In [ ]:
landcover_in_dir = '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/MCD12C1/Original/'
landcover_out_dir = '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/MCD12C1/OpenShrublands_Mask/'
lc_class = 7


In [ ]:
create_land_mask(landcover_in_dir, landcover_out_dir, lc_class)

In [2]:
#ALL DIRECTORIES USED FOR PARAMETER RASTER CALCULATION

grasslands_mask_dir = '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/MCD12C1/Grasslands_Mask/'
croplands_mask_dir = '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/MCD12C1/Croplands_Mask/'
cropnatveg_mask_dir = '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/MCD12C1/CropNatVeg_Mask/'
savannas_mask_dir = '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/MCD12C1/Savannas_Mask/'
woodysavannas_mask_dir = '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/MCD12C1/WoodySavannas_Mask/'
evergreenbroad_mask_dir = '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/MCD12C1/EvergreenBroad_Mask/'
deciduousbroad_mask_dir = '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/MCD12C1/DeciduousBroad_Mask/'
openshrublands_mask_dir = '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/MCD12C1/OpenShrublands_Mask/'
closedshrublands_mask_dir = '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/MCD12C1/ClosedShrublands_Mask/'
barren_mask_dir = '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/MCD12C1/Barren_Mask/'

grasslands_masked_out_dir = '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/VariablesByLandCover/Variables_Grasslands/'
croplands_masked_out_dir = '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/VariablesByLandCover/Variables_Croplands/'
cropnatveg_masked_out_dir = '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/VariablesByLandCover/Variables_CropNatVeg/'
savannas_masked_out_dir = '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/VariablesByLandCover/Variables_Savannas/' 
woodysavannas_masked_out_dir = '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/VariablesByLandCover/Variables_WoodySavannas/'
evergreenbroad_masked_out_dir = '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/VariablesByLandCover/Variables_EvergreenBroad/'
deciduousbroad_masked_out_dir = '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/VariablesByLandCover/Variables_DeciduousBroad/'
openshrublands_masked_out_dir = '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/VariablesByLandCover/Variables_OpenShrublands/'
closedshrublands_masked_out_dir = '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/VariablesByLandCover/Variables_ClosedShrublands/'
barren_masked_out_dir = '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/VariablesByLandCover/Variables_Barren/'

all_dirs = ['/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/CHIRPS/Resampled/', '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/CHIRTS/Dekads/', '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/eMODIS_NDVI/Resampled/', '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/FLDAS_SM/Dekads/', '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/Hobbins_ET/Resampled/', '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/LST/Resampled/']

# Multiply a chosen land cover mask by all parameters for each year

In [ ]:
def rastercalc_lcmask(lcmask_dir, param_dirs, out_dir):
    """
    This function iterates through all parameter files and multiplies the land cover mask for a certain year
    to create output rasters of each parameter masked by land cover type
    Note: must change the out name of the land cover in the out file name 
    """
    lcfiles=np.array(sorted(os.listdir(lcmask_dir)))
    
    
    for yr in range(2002,2017):
        lc_yr = [file for file in lcfiles if str(yr) in file]
        for lc in lc_yr:
            lc_mask = rasterio.open(lcmask_dir + lc)
            for folder in param_dirs:
                var_yr = [file for file in sorted(os.listdir(folder)) if str(yr) in file]
                for var in var_yr:
                    var_rast = rasterio.open(folder+var)
                    meta = var_rast.meta
                    var_lcmask = var_rast.read(1)*lc_mask.read(1)
                    with rasterio.open(out_dir + 'openshrublands_' + var, 'w', **meta) as dst:
                            dst.write(var_lcmask, 1)
                            print(var)

In [ ]:
rastercalc_lcmask(openshrublands_mask_dir, all_dirs, openshrublands_masked_out_dir)

# Rasterstats with Admin Boundaries

In [3]:
def zone_stat(raster, band, polygon, stats):
    """
    This function will calculate the zonal stats for each polygon within a raster
    requires gpd_df, raster, object and nodata value
    
    Args: raster = input raster
          band = band of raster
          polygon = polygons to calc zonal stats 
          stats = stat to calculate as string
    """
    
    band = raster.read(band)
    zone_stat = zonal_stats(polygon, band, affine=raster.meta['transform'], nodata = np.nan, stats = stats)
    return zone_stat

In [4]:
def var_poly_join(in_dir, gdf):
    files=np.array(sorted(os.listdir(in_dir)))
    tifs = pd.Series(files).str.contains('.tif')
    files = files[tifs]
    
    for filename in files:
        print(filename)
        raster = rasterio.open(in_dir+filename)
        stats = zone_stat(raster, 1, gdf, 'mean')
        print((list(stats))[-1])
        name = os.path.splitext(os.path.basename(filename))[0]
        gdf['Mean'+ "_" + name] = gpd.GeoDataFrame.from_dict(stats)
    return gdf

In [5]:
lcmasked_in_dir = openshrublands_masked_out_dir 

In [6]:
adminbds = '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/AdminBoundaries/Africa_zones_2019/g2008_af_1.shp'
bndry = gpd.read_file(adminbds)

ea_adminbds = '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/AdminBoundaries/gadm36_EastAfrica.shp'
ea_bndry = gpd.read_file(ea_adminbds)

oromia = '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/AdminBoundaries/Ethiopia/oromia.shp'
oromia_bndry = gpd.read_file(oromia)

In [7]:
output_shp = var_poly_join(lcmasked_in_dir, oromia_bndry)

openshrublands_SM_200207_01.tif
{'mean': 0.1583307683467865}
openshrublands_SM_200207_02.tif
{'mean': 0.15893233113172578}
openshrublands_SM_200207_03.tif
{'mean': 0.15707631481856835}
openshrublands_SM_200208_01.tif
{'mean': 0.15625227660667607}
openshrublands_SM_200208_02.tif
{'mean': 0.15453624398243138}
openshrublands_SM_200208_03.tif
{'mean': 0.1539948935188898}
openshrublands_SM_200209_01.tif
{'mean': 0.1719293648876795}
openshrublands_SM_200209_02.tif
{'mean': 0.24290931188478704}
openshrublands_SM_200209_03.tif
{'mean': 0.25358341488896347}
openshrublands_SM_200210_01.tif
{'mean': 0.31168175579571145}
openshrublands_SM_200210_02.tif
{'mean': 0.27468805923694517}
openshrublands_SM_200210_03.tif
{'mean': 0.29727657539088553}
openshrublands_SM_200211_01.tif
{'mean': 0.2509119743254127}
openshrublands_SM_200211_02.tif
{'mean': 0.21720564946895693}
openshrublands_SM_200211_03.tif
{'mean': 0.18370321465701592}
openshrublands_SM_200212_01.tif
{'mean': 0.1839627231039652}
openshrubland

{'mean': 0.34224099502330874}
openshrublands_SM_200605_02.tif
{'mean': 0.3574493465627112}
openshrublands_SM_200605_03.tif
{'mean': 0.33688094412408226}
openshrublands_SM_200606_01.tif
{'mean': 0.2501341180830467}
openshrublands_SM_200606_02.tif
{'mean': 0.21109674798279274}
openshrublands_SM_200606_03.tif
{'mean': 0.18211235760188685}
openshrublands_SM_200607_01.tif
{'mean': 0.16902504897699122}
openshrublands_SM_200607_02.tif
{'mean': 0.16422476310555528}
openshrublands_SM_200607_03.tif
{'mean': 0.16008103911469623}
openshrublands_SM_200608_01.tif
{'mean': 0.16137476992316363}
openshrublands_SM_200608_02.tif
{'mean': 0.1610615238910768}
openshrublands_SM_200608_03.tif
{'mean': 0.16021734476089478}
openshrublands_SM_200609_01.tif
{'mean': 0.15851924950029792}
openshrublands_SM_200609_02.tif
{'mean': 0.16416870384681514}
openshrublands_SM_200609_03.tif
{'mean': 0.242452604378142}
openshrublands_SM_200610_01.tif
{'mean': 0.29064359388700345}
openshrublands_SM_200610_02.tif
{'mean': 0.26

{'mean': 0.1685957762858142}
openshrublands_SM_201002_03.tif
{'mean': 0.1637895262759665}
openshrublands_SM_201003_01.tif
{'mean': 0.328591367472773}
openshrublands_SM_201003_02.tif
{'mean': 0.2616503569095031}
openshrublands_SM_201003_03.tif
{'mean': 0.2378261659456336}
openshrublands_SM_201004_01.tif
{'mean': 0.28952860378700757}
openshrublands_SM_201004_02.tif
{'mean': 0.33263419698113983}
openshrublands_SM_201004_03.tif
{'mean': 0.37964766310608905}
openshrublands_SM_201005_01.tif
{'mean': 0.34229056666726654}
openshrublands_SM_201005_02.tif
{'mean': 0.3542595855567766}
openshrublands_SM_201005_03.tif
{'mean': 0.33510558631109155}
openshrublands_SM_201006_01.tif
{'mean': 0.25423872406068054}
openshrublands_SM_201006_02.tif
{'mean': 0.2032517214184222}
openshrublands_SM_201006_03.tif
{'mean': 0.17900642396315283}
openshrublands_SM_201007_01.tif
{'mean': 0.16692220030919366}
openshrublands_SM_201007_02.tif
{'mean': 0.16298795750607614}
openshrublands_SM_201007_03.tif
{'mean': 0.17149

{'mean': 0.1570306856185198}
openshrublands_SM_201401_03.tif
{'mean': 0.15432796068489552}
openshrublands_SM_201402_01.tif
{'mean': 0.15236587189137935}
openshrublands_SM_201402_02.tif
{'mean': 0.15161822885274887}
openshrublands_SM_201402_03.tif
{'mean': 0.15188562348484994}
openshrublands_SM_201403_01.tif
{'mean': 0.15011032707989216}
openshrublands_SM_201403_02.tif
{'mean': 0.2160132009536028}
openshrublands_SM_201403_03.tif
{'mean': 0.22413598597049714}
openshrublands_SM_201404_01.tif
{'mean': 0.24931667782366276}
openshrublands_SM_201404_02.tif
{'mean': 0.22984099350869655}
openshrublands_SM_201404_03.tif
{'mean': 0.1896496258676052}
openshrublands_SM_201405_01.tif
{'mean': 0.340126658976078}
openshrublands_SM_201405_02.tif
{'mean': 0.372577990591526}
openshrublands_SM_201405_03.tif
{'mean': 0.2672739583998919}
openshrublands_SM_201406_01.tif
{'mean': 0.20296660028398036}
openshrublands_SM_201406_02.tif
{'mean': 0.17992552258074285}
openshrublands_SM_201406_03.tif
{'mean': 0.16672

{'mean': 33.59490613239567}
openshrublands_Tmax_200307_03.tif
{'mean': 34.42410683050388}
openshrublands_Tmax_200308_01.tif
{'mean': 32.561592520737065}
openshrublands_Tmax_200308_02.tif
{'mean': 33.045487566692074}
openshrublands_Tmax_200308_03.tif
{'mean': 34.00179439637719}
openshrublands_Tmax_200309_01.tif
{'mean': 33.86278701410061}
openshrublands_Tmax_200309_02.tif
{'mean': 35.35708701901319}
openshrublands_Tmax_200309_03.tif
{'mean': 35.62085965784585}
openshrublands_Tmax_200310_01.tif
{'mean': 34.039337949054996}
openshrublands_Tmax_200310_02.tif
{'mean': 34.47857298502108}
openshrublands_Tmax_200310_03.tif
{'mean': 34.894009939054165}
openshrublands_Tmax_200311_01.tif
{'mean': 33.730895019159085}
openshrublands_Tmax_200311_02.tif
{'mean': 33.0700938992384}
openshrublands_Tmax_200311_03.tif
{'mean': 33.19371228101777}
openshrublands_Tmax_200312_01.tif
{'mean': 32.52553567653749}
openshrublands_Tmax_200312_02.tif
{'mean': 33.746264015756005}
openshrublands_Tmax_200312_03.tif
{'m

...

{'mean': 118.98600921630859}
openshrublands_rs_ndvi.2014.031.tif
{'mean': 119.502516746521}
openshrublands_rs_ndvi.2014.032.tif
{'mean': 120.97927742004394}
openshrublands_rs_ndvi.2014.033.tif
{'mean': 123.1939640045166}
openshrublands_rs_ndvi.2014.041.tif
{'mean': 125.08529891967774}
openshrublands_rs_ndvi.2014.042.tif
{'mean': 126.09842624664307}
openshrublands_rs_ndvi.2014.043.tif
{'mean': 126.62275848388671}
openshrublands_rs_ndvi.2014.051.tif
{'mean': 127.33196468353272}
openshrublands_rs_ndvi.2014.052.tif
{'mean': 127.92720546722413}
openshrublands_rs_ndvi.2014.053.tif
{'mean': 127.11258659362792}
openshrublands_rs_ndvi.2014.061.tif
{'mean': 124.92907218933105}
openshrublands_rs_ndvi.2014.062.tif
{'mean': 122.38290615081787}
openshrublands_rs_ndvi.2014.063.tif
{'mean': 120.137251663208}
openshrublands_rs_ndvi.2014.071.tif
{'mean': 118.75003089904786}
openshrublands_rs_ndvi.2014.072.tif
{'mean': 118.3070749282837}
openshrublands_rs_ndvi.2014.073.tif
{'mean': 118.38107566833496}
op

{'mean': 74.30943996150319}
openshrublands_rs_pet_200309.tif
{'mean': 81.05065517890745}
openshrublands_rs_pet_200310.tif
{'mean': 74.65251308534204}
openshrublands_rs_pet_200311.tif
{'mean': 55.394549672196554}
openshrublands_rs_pet_200312.tif
{'mean': 45.81249804613067}
openshrublands_rs_pet_200313.tif
{'mean': 53.04285998460723}
openshrublands_rs_pet_200314.tif
{'mean': 58.52299015696456}
openshrublands_rs_pet_200315.tif
{'mean': 57.18262267694241}
openshrublands_rs_pet_200316.tif
{'mean': 62.368845311606805}
openshrublands_rs_pet_200317.tif
{'mean': 66.43498546321217}
openshrublands_rs_pet_200318.tif
{'mean': 69.39110081370283}
openshrublands_rs_pet_200319.tif
{'mean': 68.386945771008}
openshrublands_rs_pet_200320.tif
{'mean': 72.68494936315025}
openshrublands_rs_pet_200321.tif
{'mean': 86.68267264017244}
openshrublands_rs_pet_200322.tif
{'mean': 74.52557782429021}
openshrublands_rs_pet_200323.tif
{'mean': 69.88543682563596}
openshrublands_rs_pet_200324.tif
{'mean': 85.788561192954

{'mean': 52.89924249416444}
openshrublands_rs_pet_200636.tif
{'mean': 55.71515804383813}
openshrublands_rs_pet_200701.tif
{'mean': 58.18587176005045}
openshrublands_rs_pet_200702.tif
{'mean': 57.31929633730934}
openshrublands_rs_pet_200703.tif
{'mean': 67.07063211713519}
openshrublands_rs_pet_200704.tif
{'mean': 61.55419667561849}
openshrublands_rs_pet_200705.tif
{'mean': 65.637388138544}
openshrublands_rs_pet_200706.tif
{'mean': 54.34647714524042}
openshrublands_rs_pet_200707.tif
{'mean': 70.42013922191802}
openshrublands_rs_pet_200708.tif
{'mean': 67.27097120739165}
openshrublands_rs_pet_200709.tif
{'mean': 67.07279087248303}
openshrublands_rs_pet_200710.tif
{'mean': 65.78196652730306}
openshrublands_rs_pet_200711.tif
{'mean': 41.67688469659714}
openshrublands_rs_pet_200712.tif
{'mean': 53.10966718764532}
openshrublands_rs_pet_200713.tif
{'mean': 47.81131717136928}
openshrublands_rs_pet_200714.tif
{'mean': 52.25868343171619}
openshrublands_rs_pet_200715.tif
{'mean': 57.99120221819196

openshrublands_rs_pet_201026.tif
{'mean': 71.32398514125659}
openshrublands_rs_pet_201027.tif
{'mean': 63.73869008603303}
openshrublands_rs_pet_201028.tif
{'mean': 61.022279158882476}
openshrublands_rs_pet_201029.tif
{'mean': 55.56019119594408}
openshrublands_rs_pet_201030.tif
{'mean': 49.787993721340015}
openshrublands_rs_pet_201031.tif
{'mean': 57.86140325795049}
openshrublands_rs_pet_201032.tif
{'mean': 55.60003836258598}
openshrublands_rs_pet_201033.tif
{'mean': 49.86094806505286}
openshrublands_rs_pet_201034.tif
{'mean': 55.76005288828974}
openshrublands_rs_pet_201035.tif
{'mean': 56.36782803742782}
openshrublands_rs_pet_201036.tif
{'mean': 59.45950118355129}
openshrublands_rs_pet_201101.tif
{'mean': 55.602676222059465}
openshrublands_rs_pet_201102.tif
{'mean': 56.47214135064019}
openshrublands_rs_pet_201103.tif
{'mean': 70.5904283311632}
openshrublands_rs_pet_201104.tif
{'mean': 64.88560028076172}
openshrublands_rs_pet_201105.tif
{'mean': 63.48278223673503}
openshrublands_rs_pet_

{'mean': 77.4040265083313}
openshrublands_rs_pet_201423.tif
{'mean': 80.5764552116394}
openshrublands_rs_pet_201424.tif
{'mean': 85.14281148910523}
openshrublands_rs_pet_201425.tif
{'mean': 77.47124357223511}
openshrublands_rs_pet_201426.tif
{'mean': 64.00530624389648}
openshrublands_rs_pet_201427.tif
{'mean': 64.82817649841309}
openshrublands_rs_pet_201428.tif
{'mean': 41.674520301818845}
openshrublands_rs_pet_201429.tif
{'mean': 46.47747459411621}
openshrublands_rs_pet_201430.tif
{'mean': 47.073487281799316}
openshrublands_rs_pet_201431.tif
{'mean': 50.96494045257568}
openshrublands_rs_pet_201432.tif
{'mean': 48.70063991546631}
openshrublands_rs_pet_201433.tif
{'mean': 46.31044158935547}
openshrublands_rs_pet_201434.tif
{'mean': 50.89518928527832}
openshrublands_rs_pet_201435.tif
{'mean': 50.97591705322266}
openshrublands_rs_pet_201436.tif
{'mean': 58.112485027313234}
openshrublands_rs_pet_201501.tif
{'mean': 54.896068000793456}
openshrublands_rs_pet_201502.tif
{'mean': 63.3755614280

In [8]:
output_df = pd.DataFrame(output_shp.drop(columns='geometry'))

In [9]:
output_df.head()

,GID_0,NAME_0,GID_1,NAME_1,VARNAME_1,NL_NAME_1,TYPE_1,ENGTYPE_1,CC_1,HASC_1,...,Mean_openshrublands_rs_pet_201627,Mean_openshrublands_rs_pet_201628,Mean_openshrublands_rs_pet_201629,Mean_openshrublands_rs_pet_201630,Mean_openshrublands_rs_pet_201631,Mean_openshrublands_rs_pet_201632,Mean_openshrublands_rs_pet_201633,Mean_openshrublands_rs_pet_201634,Mean_openshrublands_rs_pet_201635,Mean_openshrublands_rs_pet_201636
0,ETH,Ethiopia,ETH.8_1,Oromia,Oromiya,None,Kilil,State,04,ET.OR,...,71.058071,54.966146,49.561074,61.920461,57.027244,57.548097,51.337713,50.693648,55.276036,64.046258


In [10]:
output_df.shape

(1, 3142)

In [17]:
output_df.to_csv('/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/Final_TS/oromia/'+'oromia_openshrublands.csv')



In [21]:
oromia_os = pd.read_csv('/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/Final_TS/oromia/oromia_openshrublands.csv')



# END

# Reassign cloud mask value before calculating stats

In [ ]:
def reassign_masks(in_dir):
    files=np.array(sorted(os.listdir(in_dir)))
    tifs = pd.Series(files).str.contains('.tif')
    files = files[tifs]

    for filename in files:
        with rasterio.open(in_dir+filename, 'r+') as ds:
            print(filename)
            a = ds.read()# read all raster values
            a[a < 0 ] = np.nan  
            ds.write(a)

In [ ]:
reassign_masks(variables_mask_in_dir)

# Trying dask delayed

In [ ]:
@dask.delayed
def read_and_mean(filename, in_dir, admin_gdf):
    """
    input: 
    in_dir - directory of masked land cover type
    filename - a raster variable for particular dekad
    admin_gdf - administrative boundary layer of Africa as geodataframe
    This function opens each raster and extract zonal mean for each variable
    of a particular land cover type
    """
    print(filename)
    raster = rasterio.open(in_dir+filename)
    mean = zone_stat(raster, 1, admin_gdf, 'mean')
    name = os.path.splitext(os.path.basename(filename))[0]
    return {'Mean'+ "_" + name : mean}

In [ ]:
files=np.array(sorted(os.listdir(in_dir)))
tifs = pd.Series(files).str.contains('.tif')
files = files[tifs]

admin_mean_list = []
for filename in files:
    zmeans = read_and_mean(filename, in_dir, bndry)
    admin_mean_list.append(zmeans)


In [ ]:
np.nanx

In [ ]:
from dask.distributed import Client
client = Client()
client

In [ ]:
client.compute(admin_mean_list,scheduler = 'processes')

In [ ]:
#var_grass_dir = '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/Variables_Croplands/'
#year_dir = '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/Variables_Grasslands/2016/'

In [ ]:
#output_df = output_df[output_df.columns.drop(list(output_df.filter(regex='2014')))] 
#if you want to reverse and remove a specific year from the appended dataframe

# Remove nan pixels from lst

In [ ]:
lst_test = '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/VariablesByLandCover/Variables_Clos/Version2/cropland_rs_chirps-v2-Copy1.0.2002.07.1.tif'

In [ ]:
lst_array = lst_test.read(1)

In [ ]:
lst_array.min()

In [ ]:
with rasterio.open(lst_test, 'r+') as ds:
    a = ds.read()# read all raster values
    #lst_array[np.where(lst_array<=0)]
    a[a < 0] = 0  #set all values not cropland as 0
    ds.write(a)

In [ ]:
lst_test2 = rasterio.open(lst_test)
lst_arr = lst_test2.read()
lst_arr.min()

In [ ]:
len(lst_arr[np.where(lst_arr<0)])

In [ ]:
show(lst_test2)

In [ ]:
lst_mask = lst_test.read_masks(1)

In [ ]:
lst_mask

In [ ]:
lst_array.shape

In [ ]:
lst_array[np.where(lst_array<=0)]

In [ ]:
np.where(lst_array<=0)

In [ ]:
with rasterio.open(lc_mask, 'r+') as ds:
    a = ds.read()  # read all raster values
    a[a!=4] = 0  #set all values not cropland as 0
    a[a==4] = 1
    ds.write(a)

In [ ]:
def create_land_mask(in_dir, out_dir, lc_num):
    files=np.array(sorted(os.listdir(in_dir)))
    
    for filename in files:
        base = os.path.splitext(filename)[0]
        print(base)
        with rasterio.open(in_dir+filename, 'r+') as ds:
            rast = ds.read()
            rast[rast!=lc_num] = np.nan
            rast[rast==lc_num] = 1
            meta = ds.meta()
            meta.update(
                dtype='float32')
            ds.write(rast)
                
                
                
#         rast = rasterio.open(in_dir+filename)
#         meta = rast.meta
#         meta.update(
#             dtype='float32')
#         #rast.close()
#         print(meta)
   
        
#         with rasterio.open(out_dir+base+'_' + str(lc_num) + '.tif', 'w', **meta) as dst:
#             #with rasterio.open(in_dir+filename) as src:
#                 data = rast.read()
#                 data[data!=lc_num] = -9999
#                 data[data==lc_num] = 1
#                 dst.write(data)